## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

import warnings
warnings.filterwarnings("ignore")

`(3) langfuase handler 설정`

In [3]:
from langfuse.langchain import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

`(4) Langfuse 클라이언트 설정`

In [4]:
from langfuse import get_client

# Langfuse 클라이언트 초기화
langfuse = get_client()

# 연결 테스트
assert langfuse.auth_check()

---

## **프롬프트 관리**

- Langfuse는 **프롬프트 CMS** 기능 제공

- **버전 관리** 기능으로 롤백과 성능 비교 가능

### 1. **프롬프트 생성**

In [5]:
# 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic",  # 프롬프트 이름
    type="text",          
    prompt="{{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "text"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)

In [6]:
# 챗 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic-chat",  # 프롬프트 이름
    type="chat",          
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다."
        },
        {
            "role": "user",
            "content": "영화 평론가로서, {{movie}}를 어떻게 생각하시나요?"
        }
    ], #type: ignore
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "chat"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)  

### **[실습]**

- text 기반 프롬프트와 chat 기반 프롬프트를 각각 구현하고, Langfuse UI에서 확인하세요. 

In [ ]:
# 텍스트 프롬프트 생성

In [ ]:
# 챗 프롬프트 생성


### 2. **프롬프트 활용**

In [10]:
# 프로덕션 버전 가져오기
prompt = langfuse.get_prompt("sports/sports-critique")

# 프롬프트 출력
# print(f"모델: {prompt.config['model']}")
# print(f"온도: {prompt.config['temperature']}")
print(f"라벨: {prompt.labels}")
print(f"태그: {prompt.tags}")
print(f"프롬프트: {prompt.prompt}")
print("-" * 100)

# 랭체인 프롬프트 출력
print(f"프롬프트: {prompt.get_langchain_prompt()}")

라벨: ['production', 'latest']
태그: []
프롬프트: {{criticLevel}} 스포츠 평론가로서, {{sports}}를 어떻게 생각하시나요?"
----------------------------------------------------------------------------------------------------
프롬프트: {criticLevel} 스포츠 평론가로서, {sports}를 어떻게 생각하시나요?"


### **[실습]**

- "movie-critic-chat" 프롬프트를 Langfuse에서 가져와서 내용을 출력합니다.

In [ ]:
# chat 프롬프트 가져오기

In [11]:
# 새로운 버전 생성
langfuse.create_prompt(
    name="movie-critic",  # 같은 이름 사용
    type="text",          
    prompt="당신은 {{criticLevel}} 영화 평론가입니다.\n\n영화 {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)

In [12]:
# 특정 버전 가져오기
prompt_v1 = langfuse.get_prompt("movie-critic", version=1)

# 프롬프트 출력
print(f"모델: {prompt_v1.config['model']}")
print(f"온도: {prompt_v1.config['temperature']}")
print(f"라벨: {prompt_v1.labels}")
print(f"프롬프트: {prompt_v1.prompt}")

모델: gpt-4.1-mini
온도: 0.7
라벨: ['production']
프롬프트: {{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?


### **[실습]**

- "movie-critic-chat" 프롬프트를 수정하고, labels 속성은 "staging"으로 커스텀 지정하세요.
- Langfuse에서 "staging" 버전을 가져와서 내용을 출력합니다.

In [ ]:
# staging 버전 생성


In [ ]:
# staging 버전 가져오기

In [14]:
prompt_v1.get_langchain_prompt()

'{criticLevel} 영화 평론가로서, {movie}를 어떻게 생각하시나요?'

In [15]:
from langchain_core.prompts import PromptTemplate

# Langchain과 통합 - 'text' 프롬프트
langchain_prompt = PromptTemplate.from_template(
    prompt_v1.get_langchain_prompt(),
    metadata={"langfuse_prompt": prompt_v1},
)

print(langchain_prompt.format(criticLevel="비평가", movie="인셉션"))

비평가 영화 평론가로서, 인셉션를 어떻게 생각하시나요?


In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화 (프롬프트 설정에서 가져온 값 사용)
model = ChatOpenAI(
    model=prompt_v1.config.get("model", "gpt-4.1-mini"),
    temperature=prompt_v1.config.get("temperature", 0.7)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # 콜백 핸들러 추가
)

# 응답 출력
print(response.content)

크리스토퍼 놀란 감독의 **<인셉션>(Inception, 2010)**은 현대 영화사에서 혁신적이고 영향력 있는 작품 중 하나로 평가받습니다. 이 영화는 단순한 액션 블록버스터를 넘어, 꿈과 현실의 경계, 인간의 무의식과 기억, 그리고 정체성에 대한 깊은 철학적 질문을 탐구합니다.

### 이야기와 주제
<인셉션>은 꿈속으로 들어가 타인의 무의식에 아이디어를 심는 ‘인셉션’이라는 독특한 개념을 중심으로 전개됩니다. 다층적인 꿈의 구조와 복잡한 서사 전개는 관객에게 끊임없는 긴장감과 몰입을 제공합니다. 동시에 현실과 환상의 경계가 모호해지는 경험은 ‘무엇이 진짜인가’라는 근본적인 물음을 제시하며, 관객 각자의 해석을 가능하게 만듭니다.

### 연출과 시각 효과
놀란 감독 특유의 치밀한 연출과 정교한 시나리오 구성은 영화의 복잡한 플롯을 효과적으로 이끌어냅니다. 특히 중력을 거스르는 도시의 변형, 시간의 왜곡 등 시각적으로도 혁신적인 장면들은 영화의 몰입도를 극대화합니다. 한스 짐머의 음악 역시 긴장감과 감정선을 강하게 뒷받침하며, 영화의 분위기를 완성하는 데 큰 역할을 합니다.

### 연기와 캐릭터
레오나르도 디카프리오를 비롯한 주연 배우들의 연기는 캐릭터의 내면적 갈등과 심리를 섬세하게 표현합니다. 특히 디카프리오가 연기한 ‘코브’는 죄책감과 상실, 구원에 대한 감정을 중심으로 한 복합적인 인물로, 관객이 감정적으로 깊이 공감할 수 있는 중심축 역할을 합니다.

### 결론
<인셉션>은 단순한 오락영화를 넘어, 관객에게 지적 도전과 감정적 울림을 동시에 제공하는 작품입니다. 복잡한 구조 속에서도 명확한 주제를 전달하며, 영화 예술의 새로운 가능성을 보여준 이 작품은 앞으로도 오랫동안 회자될 클래식으로 자리매김할 것입니다. 영화산업 전반에 미친 영향과 함께, 현대 서사 영화의 한 획을 그은 중요한 작품으로 보아 마땅합니다.


### **[실습]**

- 앞에서 직접 정의한 text 기반 프롬프트를 가져와서, LangChain과 통합하여 Tracing 추적을 실행합니다. 
- 실행 결과를 Langfuse UI에서 확인하세요. 

In [ ]:
# 여기에 코드를 추가하세요.

In [18]:
# 특정 라벨 가져오기
prompt_staging = langfuse.get_prompt("movie-critic-chat", label="latest")  # production, latest

# 프롬프트 출력
print(f"모델: {prompt_staging.config['model']}")
print(f"온도: {prompt_staging.config['temperature']}")
print(f"라벨: {prompt_staging.labels}")
print(f"프롬프트: {prompt_staging.prompt}")

모델: gpt-4.1-mini
온도: 0.7
라벨: ['production', 'latest']
프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다.'}, {'type': 'message', 'role': 'user', 'content': '영화 평론가로서, {{movie}}를 어떻게 생각하시나요?'}]


In [19]:
from langchain_core.prompts import ChatPromptTemplate

# Langchain과 통합 - 'chat' 프롬프트
langchain_prompt = ChatPromptTemplate.from_messages(
    prompt_staging.get_langchain_prompt(type="chat"),
)
langchain_prompt.metadata = {"langfuse_prompt": prompt_staging}

print(langchain_prompt.format(criticLevel="비평가", movie="인셉션"))

System: 당신은 비평가 영화 평론가입니다.
Human: 영화 평론가로서, 인셉션를 어떻게 생각하시나요?


In [20]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화 (프롬프트 설정에서 가져온 값 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.7)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # 콜백 핸들러 추가
)

# 응답 출력
print(response.content)

크리스토퍼 놀란 감독의 『인셉션』은 현대 영화사에서 매우 독창적이고 영향력 있는 작품으로 평가받습니다. 이 영화는 꿈과 현실의 경계를 탐구하며, 복잡한 내러티브 구조와 시각적 효과를 통해 관객에게 깊은 몰입감을 선사합니다.

특히, 놀란 특유의 정교한 각본과 한스 짐머의 강렬한 음악, 그리고 레오나르도 디카프리오를 비롯한 배우들의 뛰어난 연기가 조화를 이루어 영화의 긴장감과 감정선을 효과적으로 전달합니다. 꿈 속의 세계를 시각적으로 구현한 장면들은 기술적 완성도뿐만 아니라 철학적 질문을 던지는 데에도 성공적입니다.

다만, 일부 관객에게는 복잡한 플롯과 다층적인 시간 구조가 이해하기 어려울 수 있다는 점이 단점으로 지적되기도 합니다. 하지만 이러한 복잡성 자체가 이 작품의 매력 중 하나로, 여러 번의 관람을 통해 새로운 해석과 발견을 가능하게 합니다.

종합적으로, 『인셉션』은 장르의 경계를 넘나들며 상상력과 지성을 자극하는 작품으로서, 현대 SF 스릴러 영화의 중요한 이정표라 할 수 있습니다.


### **[실습]**

- 앞에서 직접 정의한 chat 기반 프롬프트를 가져와서, LangChain과 통합하여 Tracing 추적을 실행합니다. 
- 실행 결과를 Langfuse UI에서 확인하세요. 

In [ ]:
# 여기에 코드를 추가하세요.